In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
#matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = ''
sys.path.append(os.path.abspath('..'))
from modules import io
import modules.vascular_data as sv
import modules.sv_image as sv_image
from medpy.metric.binary import dc,hd,assd
from tqdm import tqdm

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import tensorflow as tf

In [2]:
im_fn = './UQ/user_data/X.npy'

X = np.load(im_fn,allow_pickle=True)
print(X.shape)

(308, 160, 160)


In [3]:
Nx = X.shape[0]

In [4]:
CONFIG_FILE = '../config/uq/googlenet_c30_train300k_aug10_clean_uq3.yaml'
#CONFIG_FILE = '../config/uq/unet_c30_train300k_aug10_clean.yaml'

config      = io.load_yaml(CONFIG_FILE)
config['MODEL_DIR'] = config['MODEL_DIR'].replace('.','..')

config['DROPOUT'] = 0.3
name = config['NAME']+str(config['DROPOUT'])

SCALE = config['CROP_DIMS']/2*config['SPACING']
print(name)

googlenet_c30_train300k_aug10_clean0.3


In [5]:
import factories.model_factory as model_factory

model = model_factory.get(config)
model.load()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




pool 1 Tensor("googlenet/max_pool:0", shape=(?, 79, 79, 64), dtype=float32)
pool_2 Tensor("googlenet/max_pool_1:0", shape=(?, 39, 39, 192), dtype=float32)
pool 3 Tensor("googlenet/max_pool_2:0", shape=(?, 19, 19, 480), dtype=float32)
pool side Tensor("googlenet/avg_pool:0", shape=(?, 5, 5, 833), dtype=float32)
[None, 3200]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[None, 1024]
pool 5 Tensor("googlenet/max_pool_3:0", shape=(?, 9, 9, 833), dtype=float32)
pool final Tensor("googlenet/avg_pool_1:0", shape=(?, 3, 3, 1024), dtype=float32)
[None, 9216]
[None, 

In [6]:
import factories.preprocessor_factory as prepro_factory

preprocessor = prepro_factory.get(config)

In [7]:
import factories.postprocessor_factory as postpro_factory

postprocessor = postpro_factory.get(config)

In [8]:
Xnorm = np.array([preprocessor(x) for x in X])
print(Xnorm.shape)

(308, 160, 160, 1)


In [ ]:
Nx = X.shape[0]
Nc = 25
C  = np.zeros((Nx,Nc,34,2))

In [ ]:
#gen contour samples
for i in tqdm(range(Nx)):
    x = Xnorm[i][np.newaxis,:,:,:]
    for j in range(Nc):
        #model.sample()
        yhat = model.predict(x)[0]
        C[i,j] = postprocessor(yhat)

100%|██████████| 308/308 [02:16<00:00,  2.26it/s]


In [ ]:
W = 160
SPACING = 0.029
ORIGIN = 0

rows = []

for i in tqdm(range(Nx)):
    for j in range(Nc):
        dd = []
        rr = []
        
        c = C[i,j]
        seg   = sv.contourToSeg(c,[ORIGIN]*2,[W]*2,[SPACING]*2)
        
        for k in range(Nc):
            if j==k: continue
            c2 = C[i,k]
            seg_t = sv.contourToSeg(c2,[ORIGIN]*2,[W]*2,[SPACING]*2)

            r = np.sqrt(np.sum(seg)*1.0/np.pi)*SPACING
            d = dc(seg,seg_t)
            h = hd(seg,seg_t)*SPACING
            a = assd(seg,seg_t)*SPACING
            
            data = {
                "row":i,
                "ref":j,
                "comp":k,
                "DICE":d,
                "HD":h,
                "ASSD":a,
                "radius":r
            }
        
            rows.append(data)

  3%|▎         | 9/308 [00:43<24:28,  4.91s/it]

In [ ]:
df = pd.DataFrame(rows)
df.to_csv('./UQ/user_data/{}.csv'.format(name))

In [ ]:
df = df.loc[df['HD']<1.5]
df = df.loc[df['ASSD']<1.5]

plt.figure()
plt.plot(df['radius'],df['DICE'],linestyle='',marker='o')
plt.show()
plt.close()

plt.figure()
plt.plot(df['radius'],df['HD'],linestyle='',marker='o')
plt.show()
plt.close()

plt.figure()
plt.plot(df['radius'],df['ASSD'],linestyle='',marker='o')
plt.show()
plt.close()